In [9]:
import sys
import os

# 현재 파일의 디렉토리 위치를 기준으로 경로 설정
current_dir = os.path.dirname(os.path.abspath('__file__')) 
nlp_utils_dir = os.path.abspath(os.path.join(current_dir, '..', 'NLP_utils'))

# NLP_utils 디렉토리를 sys.path에 추가
sys.path.append(nlp_utils_dir)

In [10]:
from NLP_utils import *
import time
import os
from selenium import webdriver

# 링크 추출

In [2]:
# 파일로부터 link 읽어오기
txt_path = "total_books_url.txt"

links = txt_to_list(txt_path)

# Webdriver configuration
options = get_chrome_options()
options.arguments   # 옵션 확인

['--window-size=1920,1080',
 '--headless',
 '--disable-logging',
 '--disable-images']

In [3]:
with open("urls.txt", "r") as f:
    txt = f.readlines()

txt_l = []
for line in txt:
    txt_l.append(line.strip())
print(len(txt_l))

154944


In [4]:
sub_links_ls = []

for i in range(1, 13):
    if i != 12:
        sub_links_ls.append(links[i*20:(i+1)*20])
    else:
        sub_links_ls.append(links[i*20:])

for sub_link in sub_links_ls:
    print(len(sub_link))

20
20
20
20
20
20
20
20
20
20
20
9


In [5]:
with open("urls.txt", "r") as f:
    dump = f.readlines()

txt_dump = []
for d in dump:
    txt_dump.append(d.strip())

print(len(txt_dump))
txt_dump = list(set(txt_dump))
print(len(txt_dump))

154944
154944


In [6]:
# a (append) 옵션을 주어서 덧붙여쓰기
with open("urls.txt", "w") as f:
    for url in txt_dump:
        f.write(url + "\n")

In [7]:
comp = "https://db.itkc.or.kr/dir/item?itemId=BT#/dir/node?dataId=ITKC_BT_1453A_0770_040"
print(len(comp))

80


In [7]:
max_depth = 3
last_links = []

for j in range(2, 12):
    print(f"------------------------- {j+1}th work start -------------------------")
    print()
    t1 = time.time()
    for i, link in enumerate(sub_links_ls[j], start=1):
        options = get_chrome_options()
        driver = webdriver.Chrome(options=options)

        print(f"------------------------- {(j+1)*20+i}/{len(links)} -------------------------")
        print()

        # 재귀적으로 링크 방문
        recursive_visit_links(driver, link, max_depth, last_links, str(i))
        t2 = time.time()
        print(f"-------------------- Total Elapsed time: {t2 - t1:.2f}s --------------------")
        print()

    flat_links = double_to_single(last_links)
    
    # 확실한 end-link만 추출
    flat_links = list(set(flat_links))
    
    flat_links_real = []
    for link in flat_links:
        if len(link) > len(comp):
            flat_links_real.append(link)

    # a (append) 옵션을 주어서 덧붙여쓰기
    with open("urls.txt", "a") as f:
        for url in flat_links_real:
            f.write(url + "\n")
    print()
    print(f"------------------------- Done {j+1}th work -------------------------")

------------------------- 3th work start -------------------------

------------------------- 61/249 -------------------------

-------------------- 1-157|1-1|1 => Elapsed time: 6.09s --------------------
-------------------- 1-157|1 => Elapsed time: 17.07s --------------------
-------------------- 1-157|2-1|1 => Elapsed time: 6.08s --------------------
-------------------- 1-157|2 => Elapsed time: 23.15s --------------------
-------------------- 1-157|3-1|1 => Elapsed time: 6.12s --------------------
-------------------- 1-157|3 => Elapsed time: 29.27s --------------------
-------------------- 1-157|4-2|1 => Elapsed time: 6.31s --------------------
-------------------- 1-157|4-2|2 => Elapsed time: 9.52s --------------------
-------------------- 1-157|4 => Elapsed time: 38.79s --------------------
-------------------- 1-157|5-2|1 => Elapsed time: 6.33s --------------------
-------------------- 1-157|5-2|2 => Elapsed time: 9.53s --------------------
-------------------- 1-157|5 => Elaps

# 웹 스크래핑

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, WebDriverException, TimeoutException

import concurrent.futures as cf
from concurrent.futures import TimeoutError, CancelledError
from functools import partial

import pandas as pd

In [3]:
# url 불러오기
with open("urls.txt", "r") as f:
    txt = f.readlines()

txt_l = []
for line in txt:
    txt_l.append(line.strip())
print(len(txt_l))

154944


In [4]:
# XPATH 저장

path = r'//*[@id="content"]/section/div[1]/h3'   # 전체 path 
title = r'//*[@id="content"]/section/div[2]'     # title
text_body = r'//*[@id="content"]/section/div[4]/div' # body 전문
text_desc = r'//*[@id="content"]/section/div[5]/dl' # text 설명 각주

In [5]:
# worker 함수를 래핑하여 새로운 함수 생성
options = get_chrome_options()
xpath = [path, title, text_body, text_desc]

# wrapper function
wrapped_worker = partial(worker, options=options, xpath=xpath)

# 가용 가능한 Thread의 절반만 사용함
num_workers = 25    # i5-1335U의 최대 스레드 수

In [6]:
iter_num = len(txt_l) // 1000 + 1

url_list = []
for i in range(iter_num):
    if i != iter_num:
        url_list.append(txt_l[i*1000:(i+1)*1000])
    else:
        url_list.append(txt_l[i*1000:])

In [11]:
from tqdm import tqdm
import logging

# 로거 설정
logging.basicConfig(level=logging.ERROR)

start = 136  # 이 부분을 설정해줘야 함
# 시작 시간 측정
end = 137
s = time.time()
for i, url_batch in enumerate(url_list[start:end]):
    print(f"------------------------- {start + i+1}th / {len(url_list)}th work start -------------------------")
    print()
    df = pd.DataFrame({"path": [],
                "title": [],
                "text_body": [],
                "text_desc": [],
                "url": []})
    
    chunk_size = num_workers # 한 청크의 크기
    batch_size = len(url_batch) # batch size
    num_chunks = batch_size // chunk_size + (1 if batch_size % chunk_size > 0 else 0)  # 총 청크 수

    # Scraping
    result_list = []

    with cf.ThreadPoolExecutor(num_workers) as executor:
        progress_bar = tqdm(total=batch_size)
        try:
            for j in range(num_chunks):
                chunk_start = j * chunk_size
                chunk_end = min((j + 1) * chunk_size, len(url_batch))
                chunk_urls = url_batch[chunk_start:chunk_end]

                try:
                    futures = [executor.submit(wrapped_worker, url) for url in chunk_urls]

                    for future in cf.as_completed(futures, timeout=60):
                        progress_bar.update()
                        try:
                            result = future.result()
                            result_list.append(result)
                        except TimeoutException as exception:
                            logging.error("TimeoutException occurred...")
                        except NoSuchElementException as exception2:
                            logging.error("NoSuchElementException occurred...")
                        except WebDriverException as exception3:
                            logging.error("WebDriverException occurred...")
                except TimeoutError as exception4:
                    logging.error("TimeoutError occurred...")
                    # TimeoutError 발생 시 실행 중인 Future 객체들을 취소
                    for future in futures:
                        future.cancel()
        finally:
            progress_bar.close()
    
    # 1,000개 내외 행의 데이터프레임 생성
    # 오류가 있을 시, 1,000개보다 적음
    df_add = pd.DataFrame(result_list)
    # 기존 데이터프레임과 병합

    df = pd.concat([df, df_add], axis=0, ignore_index=True)
    df.to_csv(f"classic_literature_{start + i+1}.csv", index=False)
    # 종료 시간 측정
    e = time.time()
    print(f"Total Elapsed time: {e-s:.2f}s")
    print()
    print(f"------------------------- Done {start + i+1}th work -------------------------")

------------------------- 137th / 155th work start -------------------------



  5%|▌         | 50/1000 [00:41<04:08,  3.83it/s] ERROR:root:NoSuchElementException occurred...
ERROR:root:NoSuchElementException occurred...
100%|██████████| 1000/1000 [14:02<00:00,  1.19it/s]


Total Elapsed time: 842.46s

------------------------- Done 137th work -------------------------


성공!

In [12]:
csv_list = []
for i in range(121, 156):
    path = f"classic_literature_{i}.csv"
    csv = pd.read_csv(path)
    print(i, csv.shape)
    csv_list.append(csv)

121 (997, 5)
122 (998, 5)
123 (997, 5)
124 (998, 5)
125 (999, 5)
126 (996, 5)
127 (970, 5)
128 (999, 5)
129 (999, 5)
130 (996, 5)
131 (995, 5)
132 (999, 5)
133 (998, 5)
134 (991, 5)
135 (972, 5)
136 (1000, 5)
137 (996, 5)
138 (983, 5)
139 (1000, 5)
140 (998, 5)
141 (1000, 5)
142 (1000, 5)
143 (1000, 5)
144 (999, 5)
145 (999, 5)
146 (1000, 5)
147 (1000, 5)
148 (1000, 5)
149 (999, 5)
150 (997, 5)
151 (950, 5)
152 (997, 5)
153 (996, 5)
154 (999, 5)
155 (942, 5)


In [22]:
df_t.shape

(154222, 5)